## This is our workspace for statistical analysis!

Add cells, do exploratory analysis, go ham.

In [2]:
# feel free to add more

import sqlite3
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm
from statsmodels.tools import eval_measures

In [3]:
# Create connection to database
conn = sqlite3.connect('../data/db/avocado_project.db')
c = conn.cursor()

In [4]:
# Print out tables in database
list_tables_cmd = "SELECT name FROM sqlite_master WHERE type='table';"
c.execute(list_tables_cmd)
conn.commit()

for row in c:
  print(row)

('demographic',)
('hab2016',)
('hab2017',)
('hab2018',)


In [23]:
# Load all tables into dataframe
query1 = "SELECT * FROM demographic"
demo_df = pd.read_sql(query1, conn)

query2 = "SELECT * FROM hab2016"
hab2016_df = pd.read_sql(query2, conn)
hab2016_df.rename(columns={'units_current_year': '2016_units_current_year',
                          'dollars_current_year': '2016_dollars_current_year',
                          'asp_current_year': '2016_asp_current_year'}, inplace=True)

query3 = "SELECT * FROM hab2017"
hab2017_df = pd.read_sql(query3, conn)
hab2017_df.rename(columns={'units_current_year': '2017_units_current_year',
                          'dollars_current_year': '2017_dollars_current_year',
                          'asp_current_year': '2017_asp_current_year'}, inplace=True)

query4 = "SELECT * FROM hab2018"
hab2018_df = pd.read_sql(query4, conn)
hab2018_df.rename(columns={'units_current_year': '2018_units_current_year',
                          'dollars_current_year': '2018_dollars_current_year',
                          'asp_current_year': '2018_asp_current_year'}, inplace=True)

In [ ]:
mean_pop = []
median_pop = []
stddev_pop = []

for index, row in demo_df.iterrows():
    pop_list = [row['pop2016'], row['pop2017'], row['pop2018']]
    means = np.mean(pop_list)
    median = np.median(pop_list)
    std_dev = np.std(pop_list)
    mean_pop.append(means)
    median_pop.append(median)
    stddev_pop.append(std_dev)
    
demo_df["Mean Population"] = mean_pop
demo_df["Median Population"] = median_pop
demo_df["Standard Deviation Population"] = stddev_pop
demo_df

#we might want to remove the rows with age 999 as its just an aggregation of all the previous rows of that region


In [26]:
# Load all tables into dataframe
# query1 = "SELECT * FROM demographic"
# demo_df = pd.read_sql(query1, conn)

query2 = "SELECT * FROM hab2016"
hab2016_raw_df = pd.read_sql(query2, conn)
hab2016_raw_df['Year'] = 2016

query3 = "SELECT * FROM hab2017"
hab2017_raw_df = pd.read_sql(query3, conn)
hab2017_raw_df['Year'] = 2017

query4 = "SELECT * FROM hab2018"
hab2018_raw_df = pd.read_sql(query4, conn)
hab2018_raw_df['Year'] = 2018

In [29]:
demo_df.describe()

,age,pop2016,pop2017,pop2018
count,1392.000000,1.392000e+03,1.392000e+03,1.392000e+03
mean,53.494253,4.594373e+05,4.624431e+05,4.653553e+05
std,104.939063,2.169294e+06,2.182227e+06,2.194852e+06
min,0.000000,2.898500e+04,2.899400e+04,2.861700e+04
25%,21.000000,1.620888e+05,1.661180e+05,1.762585e+05
50%,43.000000,2.488850e+05,2.496150e+05,2.505665e+05
75%,65.000000,2.848312e+05,2.860640e+05,2.871700e+05
max,999.000000,2.909011e+07,2.909579e+07,2.911077e+07


In [28]:
# join all three avocado data frames
hab_df = pd.concat([hab2016_raw_df, hab2017_raw_df, hab2018_raw_df], axis=1)
hab_df

,region,units_prior_year,units_current_year,dollars_prior_year,dollars_current_year,asp_prior_year,asp_current_year,Year,region,units_prior_year,...,asp_prior_year,asp_current_year,Year,region,units_prior_year,units_current_year,dollars_prior_year,dollars_current_year,asp_prior_year,asp_current_year
0,California,2.815620e+08,2.742912e+08,2.938569e+08,3.161020e+08,12.553616,14.193095,2016,California,3.163792e+08,...,16.156570,18.487848,2017,California,2.892980e+08,3.449431e+08,3.806062e+08,3.965100e+08,17.467113,15.074676
1,Great Lakes,1.565253e+08,1.585368e+08,1.746281e+08,1.920213e+08,13.370501,14.862985,2016,Great Lakes,1.828621e+08,...,17.024001,19.554727,2017,Great Lakes,1.767339e+08,2.205796e+08,2.455320e+08,2.528185e+08,18.420128,14.962134
2,Midsouth,1.353713e+08,1.470024e+08,1.532692e+08,1.749785e+08,13.567583,14.537764,2016,Midsouth,1.666395e+08,...,16.789138,18.956182,2017,Midsouth,1.626263e+08,2.020955e+08,2.206566e+08,2.370726e+08,17.740391,15.288319
3,Northeast,1.911689e+08,2.077816e+08,2.342339e+08,2.654976e+08,14.719099,15.702439,2016,Northeast,2.367107e+08,...,18.151362,20.727892,2017,Northeast,2.240129e+08,2.939438e+08,3.334003e+08,3.715844e+08,19.418093,16.582135
4,Plains,8.319278e+07,8.805425e+07,9.108528e+07,9.987437e+07,13.170314,13.988476,2016,Plains,1.011263e+08,...,16.074756,18.529716,2017,Plains,9.559421e+07,1.170123e+08,1.266354e+08,1.283651e+08,17.497439,14.276865
5,South Central,2.755884e+08,2.798747e+08,2.304405e+08,2.495642e+08,10.050216,10.873204,2016,South Central,3.231953e+08,...,12.432734,13.872400,2017,South Central,3.090627e+08,3.846433e+08,3.079581e+08,3.216136e+08,13.120511,10.926425
6,Southeast,1.498071e+08,1.755572e+08,1.646243e+08,1.968386e+08,13.199943,13.874733,2016,Southeast,2.003659e+08,...,16.061944,19.002511,2017,Southeast,1.895716e+08,2.603579e+08,2.566267e+08,2.893458e+08,17.859741,14.499490
7,West,2.782710e+08,2.821855e+08,2.785080e+08,2.989914e+08,12.025842,13.064521,2016,West,3.260830e+08,...,14.994334,16.698210,2017,West,3.081864e+08,3.753159e+08,3.665522e+08,4.001218e+08,15.696705,13.986238


In [30]:
hab_df.describe()

,units_prior_year,units_current_year,dollars_prior_year,dollars_current_year,asp_prior_year,asp_current_year,Year,units_prior_year,units_current_year,dollars_prior_year,dollars_current_year,asp_prior_year,asp_current_year,Year,units_prior_year,units_current_year,dollars_prior_year,dollars_current_year,asp_prior_year,asp_current_year
count,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000,8.000000,8.0,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000,8.000000,8.0,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000,8.000000
mean,1.939358e+08,2.016604e+08,2.025808e+08,2.242335e+08,12.832139,13.887152,2016.0,2.316702e+08,2.346961e+08,2.544891e+08,2.956927e+08,15.960605,18.228686,2017.0,2.193858e+08,2.748614e+08,2.797460e+08,2.996790e+08,17.152515,14.449535
std,7.606980e+07,7.206084e+07,6.861828e+07,7.176579e+07,1.368406,1.441004,0.0,8.377225e+07,8.354931e+07,8.153537e+07,8.960294e+07,1.692756,2.091132,0.0,7.754325e+07,9.318708e+07,8.453428e+07,9.311070e+07,1.933902,1.628512
min,8.319278e+07,8.805425e+07,9.108528e+07,9.987437e+07,10.050216,10.873204,2016.0,1.011263e+08,1.024189e+08,1.135041e+08,1.336804e+08,12.432734,13.872400,2017.0,9.559421e+07,1.170123e+08,1.266354e+08,1.283651e+08,13.120511,10.926425
25%,1.461981e+08,1.556532e+08,1.617856e+08,1.877606e+08,12.421673,13.672180,2016.0,1.788064e+08,1.850176e+08,2.130242e+08,2.529119e+08,15.795042,18.040438,2017.0,1.732070e+08,2.159586e+08,2.393131e+08,2.488821e+08,17.024511,14.204208
50%,1.738471e+08,1.916694e+08,2.025343e+08,2.232014e+08,13.185128,14.090785,2016.0,2.185383e+08,2.203551e+08,2.536511e+08,2.982023e+08,16.115663,18.742949,2017.0,2.067923e+08,2.771509e+08,2.822924e+08,3.054797e+08,17.618915,14.730812
75%,2.762590e+08,2.756871e+08,2.453024e+08,2.738711e+08,13.419771,14.619069,2016.0,3.180832e+08,3.153956e+08,3.110090e+08,3.613205e+08,16.847854,19.140565,2017.0,2.940201e+08,3.525363e+08,3.416883e+08,3.778158e+08,17.999838,15.128087
max,2.815620e+08,2.821855e+08,2.938569e+08,3.161020e+08,14.719099,15.702439,2016.0,3.260830e+08,3.317882e+08,3.574163e+08,4.020201e+08,18.151362,20.727892,2017.0,3.090627e+08,3.846433e+08,3.806062e+08,4.001218e+08,19.418093,16.582135


In [ ]:
region = []
current_units_mean = []
current_units_median = []
current_units_stddev = []

current_dollars_mean = []
current_dollars_median = []
current_dollars_stddev = []

current_asp_mean = []
current_asp_median = []
current_asp_stddev = []

for index, row in hab2016_df.iterrows(): 
    region.append(row['region'])
    for index1, row1 in hab2017_df.iterrows():
        for index2, row2 in hab2018_df.iterrows():
            current_units_list = [row['2016_units_current_year'], row1['2017_units_current_year'], row2['2018_units_current_year']]
            current_dollars_list = [row['2016_dollars_current_year'], row1['2017_dollars_current_year'], row2['2018_dollars_current_year']]
            current_asp_list = [row['2016_asp_current_year'], row1['2017_asp_current_year'], row2['2018_asp_current_year']]
            
    units_mean = np.mean(current_units_list)
    units_median = np.median(current_units_list)
    units_std_dev = np.std(current_units_list)
    current_units_mean.append(units_mean)
    current_units_median.append(units_median)
    current_units_stddev.append(units_std_dev)
            
    dollars_mean = np.mean(current_dollars_list)
    dollars_median = np.median(current_dollars_list)
    dollars_std_dev = np.std(current_dollars_list)
    current_dollars_mean.append(dollars_mean)
    current_dollars_median.append(dollars_median)
    current_dollars_stddev.append(dollars_std_dev)
            
    asp_mean = np.mean(current_asp_list)  
    asp_median = np.median(current_asp_list)
    asp_std_dev = np.std(current_asp_list)  
    current_asp_mean.append(asp_mean)
    current_asp_median.append(asp_median)
    current_asp_stddev.append(asp_std_dev)
            
combined_avo_data = {'region' : region,
                     'units_current_year_mean' : current_units_mean, 
                     'units_current_year_median' : current_units_median,
                    'units_current_year_std_dev' : current_units_stddev,
                    'dollars_current_year_mean' : current_dollars_mean, 
                     'dollars_current_year_median' : current_dollars_median,
                    'dollars_current_year_std_dev' : current_dollars_stddev,
                    'asp_current_year_mean' : current_asp_mean, 
                     'asp_current_year_median' : current_asp_median,
                    'asp_current_year_std_dev' : current_asp_stddev}

combined_avo_df = pd.DataFrame(combined_avo_data)
combined_avo_df

In [21]:
# summarize dataframe
demo_df.describe()

,age,pop2016,pop2017,pop2018
count,256.000000,256.000000,256.000000,256.000000
mean,27.500000,273738.703125,274983.199219,276028.218750
std,4.618802,64981.381363,65022.503640,65120.031945
min,20.000000,132729.000000,133290.000000,133884.000000
25%,23.750000,249830.500000,251822.250000,253082.000000
50%,27.500000,277373.000000,279592.500000,280493.000000
75%,31.250000,301072.250000,303494.750000,305040.000000
max,35.000000,412189.000000,415091.000000,416227.000000


In [19]:
# parse out rows that correspon to millenials
demo_df = demo_df.loc[(demo_df['age'] <= 35) & (demo_df['age'] >= 20)]
demo_df

# todo sum F and M together to get total population

,region,age,sex,pop2016,pop2017,pop2018
40,California,20,F,252888,249663,246589
41,California,20,M,261741,256487,253086
42,California,21,F,259769,254458,251061
43,California,21,M,268650,262335,258126
44,California,22,F,268170,264370,258629
45,California,22,M,278723,271867,266407
46,California,23,F,276876,275249,271318
47,California,23,M,289958,286732,279911
48,California,24,F,290396,281569,279531
49,California,24,M,306282,296538,293568
